In [45]:
import numpy as np
import pandas as pd

In [60]:
data_kaggle = pd.read_csv(f"F:\Desktop\暑假疯狂自学\youtube机器学习数学课（进行中）\spam-or-not-spam-dataset\spam_or_not_spam.csv")

In [47]:
data_kaggle.head()

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0


In [56]:
len(data_kaggle)

3000

In [202]:
data['label'] = data_kaggle['label']
wordlist = []
emaillist = []
elist = data_kaggle['email'].astype(str).tolist()
for e in elist:
    wordlist = e.split(' ')
    if wordlist[0] == '':
        wordlist.pop(0)
    elif wordlist[-1] == '':
            wordlist.pop()
    emaillist.append(wordlist)


In [203]:
data['email'] = pd.Series(emaillist)

In [204]:
data.head()

,email,label
0,"[date, wed, NUMBER, aug, NUMBER, NUMBER, NUMBE...",0
1,"[martin, a, posted, tassos, papadopoulos, the,...",0
2,"[man, threatens, explosion, in, moscow, thursd...",0
3,"[klez, the, virus, that, won, t, die, already,...",0
4,"[in, adding, cream, to, spaghetti, carbonara, ...",0


In [205]:
from sklearn.model_selection import train_test_split
x = data['email']
y = data['label']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.33)

## Naive Bayes

In [206]:
# create traindata and testdata
train1_data = pd.DataFrame()
test1_data = pd.DataFrame()
train1_data['email'] = x_train
train1_data['label'] = y_train
test1_data['email'] = x_test
test1_data['label'] = y_test
train_dict = {'email':pd.Series(train1_data['email'].values),'label':pd.Series(train1_data['label'].values)}
test_dict = {'email':pd.Series(test1_data['email'].values),'label':pd.Series(test1_data['label'].values)}
train_data = pd.DataFrame(train_dict)
test_data = pd.DataFrame(test_dict)

In [207]:
def conflate(adict,bdict):
    dic = {}
    for key in adict:
        if bdict.get(key):
            dic[key] = adict[key]+bdict[key]
        else:
            dic[key] = adict[key]
    for key in bdict:
        if adict.get(key):
            pass
        else:
            dic[key] = bdict[key]
    return dic

def train():
    total = 0
    numSpam = 0
    trainPositive = {}
    trainNegative = {}
    positiveTotal,negativeTotal = 0,0
    for i in range(len(train_data)):
        piece = train_data.loc[i]
        if piece.label == 1:
            numSpam += 1
        total += 1
        atuple = processEmail(piece.email,piece.label)
        positiveTotal += atuple[0]
        negativeTotal += atuple[1]
        trainPositive = conflate(trainPositive,atuple[2])
        trainNegative = conflate(trainNegative,atuple[3])
    return numSpam/total,(total-numSpam)/total,positiveTotal,negativeTotal,trainPositive,trainNegative
    
# reading words from a specific email
def processEmail(email,label):
    positiveTotal,negativeTotal = 0,0
    trainPositive = {}
    trainNegative = {}
    for word in email:
        if label == 1:
            trainPositive[word] = trainPositive.get(word,0) + 1
            positiveTotal += 1
        else:
            trainNegative[word] = trainNegative.get(word,0) + 1
            negativeTotal += 1
    return positiveTotal,negativeTotal,trainPositive,trainNegative
        
# gives the conditional probability p(B/A_x)
def conditionalEmail(email,label):
    result = 1
    for word in email:
        result *= conditionalWord(word,label)
    return result

# classifies a new email as spam or not spam
def classify(email):
    isSpam = PA * conditionalEmail(email,True)
    notSpam = PNotA * conditionalEmail(email,False)
    return 1 if isSpam>notSpam else 0
# Laplace Smoothing for the words not present in the training set
# gives the conditional probability P(B_i/A_x) with smoothing
def conditionalWord(word,label):
    if label:
        return (trainPositive.get(word,0)+alpha)/(float)(positiveTotal+alpha*numWords)
    return (trainNegative.get(word,0)+alpha)/(float)(negativeTotal+alpha*numWords)

        

In [208]:
PA,PNotA,positiveTotal,negativeTotal,trainPositive,trainNegative = train()
numWords = 50000
alpha = 1

In [209]:
print(PA)
print(PNotA)
print(positiveTotal)
print(negativeTotal)
print(trainPositive[''])
print(trainNegative[''])

0.1701492537313433
0.8298507462686567
98884
362919
182
453


In [210]:
prediction = [classify(email) for email in test_data['email'].tolist()]
test_data['prediction'] = pd.Series(prediction)
test_data['accuracy'] = pd.Series([0 for i in range(len(test_data))])
test_data['accuracy'][test_data['prediction']==test_data['label']] = 1
accuracy = sum(test_data['accuracy'].tolist())/len(test_data)
print(accuracy)

0.8818181818181818


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
